In [7]:
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, to_date, monotonically_increasing_id

spark = SparkSession.builder\
.master("local")\
.appName("Word Count")\
.getOrCreate()
#.config("spark.some.config.option", "some-value")\

In [8]:
df_src2_1 = spark.read.options(header='true', inferSchema='true').parquet("../data_source/parquet/chess_game_SAHIT.parquet")
df1 = spark.read.options(header='true', inferSchema='true').parquet("../data_source/parquet/chess_game_REVEL.parquet")

In [9]:
df_src2_2 = df_src2_1.withColumnRenamed('Date', 'UTCDate') 


df2 = df1.withColumn("Result", when((col("WhiteRatingDiff") < 0) & (col("BlackRatingDiff") > 0), "0-1")
    .when((col("WhiteRatingDiff") > 0) & (col("BlackRatingDiff") < 0), "1-0")
    .when((col("WhiteRatingDiff") == 0) & (col("BlackRatingDiff") == 0), "1/2-1/2")
    .otherwise(col("Result"))
)

df3 = df2.dropDuplicates().drop("Format").drop("AN").drop("WhiteRatingDiff").drop("BlackRatingDiff").drop("UTCTime")
df_src2_3 = df_src2_2.dropDuplicates().drop("Format").drop("Round").drop("WhiteRatingDiff")

df4 = df3.filter(
    (df2["Event"].contains("Blitz")) |
    (df2["Event"].contains("Classic")) |
    (df2["Event"].contains("Bullet"))
)

df_src2_4 = df_src2_3.filter(
    (df_src2_3["Event"].contains("Blitz")) |
    (df_src2_3["Event"].contains("Classic")) |
    (df_src2_3["Event"].contains("Bullet"))
)


df5 = df4.withColumn("Event", when(df3["Event"].contains("Blitz"), "Blitz").when(df3["Event"].contains("Classic"), "Classic").when(df3["Event"].contains("Bullet"), "Bullet").otherwise(df3["Event"]))
df_src2_5 = df_src2_4.withColumn("Event", when(df_src2_2["Event"].contains("Blitz"), "Blitz").when(df_src2_2["Event"].contains("Classic"), "Classic").when(df_src2_2["Event"].contains("Bullet"), "Bullet").otherwise(df_src2_2["Event"]))

df6 = df5.withColumn("UTCDate", to_date("UTCDate", "yyyy.MM.dd"))
df_src2_6 = df_src2_5.withColumn("UTCDate", to_date("UTCDate", "yyyy.MM.dd"))

df7 =  df6.filter(col("Result") != '*')
## Change this
df_src2_7 =  df_src2_6.filter(col("Result") != '*')


df8 = df7.withColumn("id", monotonically_increasing_id())
df_src2_8 = df_src2_7.withColumn("id", monotonically_increasing_id())


df9 = df8.select("id", "Event", "White", "Black", "Result", "UTCDate", "WhiteElo", "BlackElo", "ECO", "Opening", "Termination")
df_src2_9 = df_src2_8.select("id", "Event", "White", "Black", "Result", "UTCDate", "WhiteElo", "BlackElo", "ECO", "Opening", "Termination")


df9.show()
df_src2_9.show()

+---+-------+---------------+--------------+------+----------+--------+--------+---+--------------------+------------+
| id|  Event|          White|         Black|Result|   UTCDate|WhiteElo|BlackElo|ECO|             Opening| Termination|
+---+-------+---------------+--------------+------+----------+--------+--------+---+--------------------+------------+
|  0|Classic|     whitty1973|        DonQyf|   1-0|2016-06-30|    1580|    1571|D02|Queen's Pawn Game...|      Normal|
|  1|  Blitz|         Inorem|      leymoral|   1-0|2016-06-30|    1751|    1712|C62|Ruy Lopez: Steini...|      Normal|
|  2|Classic|     avances123| sakilaharshad|   1-0|2016-06-30|    1624|    1614|C42|Russian Game: Cla...|      Normal|
|  3|Classic|      Edprenton|   Firdaus_edo|   0-1|2016-06-30|    1787|    1842|A30|English Opening: ...|      Normal|
|  4|  Blitz|         kyoday|     BrettDale|   0-1|2016-06-30|    1945|    1900|B90|Sicilian Defense:...|Time forfeit|
|  5|  Blitz|         go4jas|    Sergei1973|   0